# Job Monitoring and Management on Explorer Cluster

This notebook teaches you how to monitor, manage, and troubleshoot your jobs on the Explorer cluster.

## Table of Contents
1. [Job Status Monitoring](#job-status)
2. [Queue Management](#queue-management)
3. [Job History and Accounting](#job-history)
4. [Resource Usage Analysis](#resource-usage)
5. [Interactive Monitoring Tools](#interactive-tools)

## 1. Job Status Monitoring <a name="job-status"></a>

### 1.1 Basic Job Checking

In [ ]:
# Essential job monitoring commands
monitoring_commands = """
# Check your current jobs
squeue -u $USER

# Check all jobs in gpu partition
squeue -p gpu

# Check specific job details
scontrol show job <JOBID>

# Check estimated start time
squeue -u $USER --start

# Check job priority
sprio -u $USER
"""

print(monitoring_commands)

### 1.2 Job Status Meanings

In [ ]:
# Job status codes explanation
status_codes = {
    'R': 'Running - Job is currently executing',
    'PD': 'Pending - Job is waiting for resources',
    'CG': 'Completing - Job is finishing up',
    'CD': 'Completed - Job finished successfully',
    'F': 'Failed - Job terminated with error',
    'CA': 'Cancelled - Job was cancelled',
    'TO': 'Timeout - Job exceeded time limit',
    'OOM': 'Out of Memory - Job ran out of memory'
}

print("Job Status Codes:")
for code, meaning in status_codes.items():
    print(f"{code:3}: {meaning}")

## 2. Queue Management <a name="queue-management"></a>

### 2.1 Understanding Queue Position

In [ ]:
# Queue analysis commands
queue_commands = """
# Check partition information
sinfo -p gpu

# Check node availability
sinfo -N -l

# Check your position in queue
squeue -u $USER -o "%.18i %.9P %.8j %.8u %.2t %.10M %.6D %R"

# Check why job is pending
squeue -j <JOBID> --start

# Check fair share priority
sshare -u $USER
"""

print(queue_commands)

### 2.2 Waiting Time Estimation

In [ ]:
import subprocess
import re
from datetime import datetime, timedelta

def estimate_wait_time():
    """Estimate waiting time for your jobs"""
    try:
        # Get job info with start time estimates
        result = subprocess.run(['squeue', '-u', '$USER', '--start'], 
                              capture_output=True, text=True)
        
        print("Your jobs with estimated start times:")
        print(result.stdout)
        
        # Parse and analyze (simplified example)
        lines = result.stdout.strip().split('\n')
        if len(lines) > 1:
            print("\nWait time analysis:")
            for line in lines[1:]:  # Skip header
                if 'N/A' not in line:
                    print(f"Job has estimated start time")
                else:
                    print(f"Job waiting for resources")
    except:
        print("Run 'squeue -u $USER --start' in terminal to check wait times")

estimate_wait_time()

## 3. Job History and Accounting <a name="job-history"></a>

### 3.1 Recent Job History

In [ ]:
# Job history commands
history_commands = """
# Check today's jobs
sacct -u $USER --starttime=today

# Check last week's jobs
sacct -u $USER --starttime=week

# Detailed job information
sacct -j <JOBID> --format=JobID,JobName,Partition,Account,AllocCPUS,State,ExitCode,Start,End,Elapsed,MaxRSS,MaxVMSize

# Check job efficiency
seff <JOBID>

# Custom format for specific info
sacct -u $USER --format=JobID,JobName,State,ExitCode,Start,End,Elapsed,MaxRSS --starttime=today
"""

print(history_commands)

### 3.2 Job Efficiency Analysis

In [ ]:
def analyze_job_efficiency():
    """Analyze job efficiency metrics"""
    efficiency_guide = """
    Job Efficiency Metrics:
    
    CPU Efficiency:
    - >90%: Excellent CPU utilization
    - 70-90%: Good utilization
    - <70%: Consider reducing CPU request
    
    Memory Efficiency:
    - >80%: Good memory utilization
    - 50-80%: Acceptable
    - <50%: Consider reducing memory request
    
    GPU Efficiency:
    - >80%: Excellent GPU utilization
    - 60-80%: Good utilization
    - <60%: Check for bottlenecks
    
    Time Efficiency:
    - Used <50% of requested time: Consider shorter time limits
    - Used >95% of requested time: Consider longer time limits
    """
    
    print(efficiency_guide)

analyze_job_efficiency()

## 4. Resource Usage Analysis <a name="resource-usage"></a>

### 4.1 Real-time Resource Monitoring

In [ ]:
# Real-time monitoring commands
realtime_commands = """
# Monitor running job resources
sstat -j <JOBID> --format=AveCPU,AvePages,AveRSS,AveVMSize,JobID

# Continuous monitoring (run in terminal)
watch -n 5 'squeue -u $USER'

# SSH to compute node (if job is running)
ssh <node-name>
htop  # Monitor CPU/memory
nvidia-smi  # Monitor GPU

# Monitor specific job on node
ssh <node-name> 'ps aux | grep <your-process>'
"""

print(realtime_commands)

### 4.2 GPU Monitoring Script

In [ ]:
# GPU monitoring script
gpu_monitor_script = '''
#!/bin/bash
# gpu_monitor.sh - Monitor GPU usage for your jobs

# Get your running jobs
JOBS=$(squeue -u $USER -h -o "%i %N" | grep -v "(None)")

if [ -z "$JOBS" ]; then
    echo "No running jobs found"
    exit 0
fi

echo "Monitoring GPU usage for your jobs:"
echo "===================================="

while IFS= read -r line; do
    JOBID=$(echo $line | awk '{print $1}')
    NODE=$(echo $line | awk '{print $2}')
    
    echo "Job $JOBID on node $NODE:"
    ssh $NODE "nvidia-smi --query-gpu=utilization.gpu,memory.used,memory.total --format=csv,noheader,nounits" 2>/dev/null || echo "Cannot connect to $NODE"
    echo "---"
done <<< "$JOBS"
'''

with open('gpu_monitor.sh', 'w') as f:
    f.write(gpu_monitor_script)

print("Created gpu_monitor.sh")
print("Make executable with: chmod +x gpu_monitor.sh")
print("Run with: ./gpu_monitor.sh")

## 5. Interactive Monitoring Tools <a name="interactive-tools"></a>

### 5.1 Job Dashboard Function

In [ ]:
import subprocess
import time
from datetime import datetime

def job_dashboard():
    """Simple job monitoring dashboard"""
    try:
        # Get current jobs
        result = subprocess.run(['squeue', '-u', '$USER'], 
                              capture_output=True, text=True)
        
        print(f"Job Dashboard - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print("=" * 60)
        
        if result.returncode == 0:
            lines = result.stdout.strip().split('\n')
            if len(lines) > 1:
                print(result.stdout)
                
                # Count job states
                running = result.stdout.count(' R ')
                pending = result.stdout.count(' PD ')
                
                print(f"\nSummary: {running} running, {pending} pending")
            else:
                print("No jobs currently in queue")
        else:
            print("Error checking jobs")
            
    except Exception as e:
        print(f"Error: {e}")
        print("Run 'squeue -u $USER' in terminal")

# Run the dashboard
job_dashboard()

### 5.2 Automated Job Alerts

In [ ]:
# Job alert script
alert_script = '''
#!/bin/bash
# job_alerts.sh - Get notified when jobs complete

# Usage: ./job_alerts.sh <jobid>

JOBID=$1

if [ -z "$JOBID" ]; then
    echo "Usage: $0 <jobid>"
    exit 1
fi

echo "Monitoring job $JOBID..."

while true; do
    STATUS=$(squeue -j $JOBID -h -o "%T" 2>/dev/null)
    
    if [ -z "$STATUS" ]; then
        echo "Job $JOBID completed at $(date)"
        
        # Check job outcome
        OUTCOME=$(sacct -j $JOBID -n -o State | head -1 | tr -d ' ')
        echo "Final status: $OUTCOME"
        
        # Show efficiency
        echo "Job efficiency:"
        seff $JOBID
        
        break
    fi
    
    echo "Job $JOBID status: $STATUS ($(date))"
    sleep 60  # Check every minute
done
'''

with open('job_alerts.sh', 'w') as f:
    f.write(alert_script)

print("Created job_alerts.sh")
print("Usage: chmod +x job_alerts.sh && ./job_alerts.sh <jobid>")

## Quick Reference

### Essential Monitoring Commands

| Command | Purpose |
|---------|----------|
| `squeue -u $USER` | Check your jobs |
| `scontrol show job <ID>` | Job details |
| `sacct -j <ID>` | Job history |
| `seff <ID>` | Job efficiency |
| `scancel <ID>` | Cancel job |
| `sinfo -p gpu` | Partition status |
| `sprio -u $USER` | Job priority |

### Troubleshooting Workflow

1. **Check job status**: `squeue -u $USER`
2. **If pending**: `squeue -j <ID> --start`
3. **If failed**: `sacct -j <ID>` and check `.err` file
4. **Check efficiency**: `seff <ID>`
5. **Adjust resources** for next submission

### Best Practices

- Monitor jobs regularly
- Check efficiency after completion
- Adjust resource requests based on usage
- Use email notifications for long jobs
- Keep job logs organized

---

**Next Steps:**
- Practice monitoring your jobs
- Set up automated alerts
- Analyze job efficiency
- Optimize resource requests